<a href="https://colab.research.google.com/github/gorkemozkaya/Data-Science-Notes/blob/master/generalized_linear_models/Poisson%20regression%20with%20Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Poisson regression with Tensorflow
*Gorkem Ozkaya*

In this notebook we will perform a Poisson regression with Google's Deep Learning library Tensorflow. Although Tensorflow is not primarily designed for traditional modeling tasks such as Poisson regression, we still can benefit from the flexibility of Tensorflow. Starting from shallow models like these, one can later obtain deeper versions of Poisson regression and other GLM's, which can handle nonlinearaties and variable interactions. 

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Generating data

In [5]:
def gen_data(N = 10000):
    """ generating sample data for the model
        y = -2 + 1.0 * X_1 - 0.5 * X_2 + 0.3 * X_3
    """
    data = np.random.uniform(-1, 1, (N, 3))
    data = sm.add_constant(data)
    data = pd.DataFrame(data, columns = ['intercept', 'Var1', 'Var2', 'Var3'])
    lam = np.exp(-2 * data['intercept'] + data['Var1'] - 0.5 * data['Var2'] + 0.3 * data['Var3'] )
    resp = np.random.poisson(lam = lam)
    data['lam'] = lam
    data['resp'] = resp
    return data

In [6]:
dtrain = gen_data()

### Stating Poisson regression as an optimization problem
Let $X$ be the design matrix, $w$ be the model coefficient vector and $y$ be the observed response. Let $\hat y(w) = \exp\left(Xw\right)$ be the estimated mean by the model. We are looking for the coefficient vector $w$ that maximizes the likelihood
$$
L(w) =  \prod_{i = 1}^N \frac{\hat y_i(w)^{y_i} e^{-\hat y_i(w)}}{y_i!},
$$
where $y = (y_i)_i$ and $\hat y = (\hat y_i)_i$.  Taking logarithms and removing the constants, this problem is equivalent to minimizing the loss function
$$
   L(w)  =  -\sum_{i=1}^N \left(y_i \log(\hat y_i(w)) - \hat y_i(w) \right).
$$


### Fitting the model with Tensorflow
Having defined the objective function, now we can set the Tensorflow model. With its automatic differentiation support, Tensorflow automatically calculates the gradients of mathematical expressions, hence can do gradient descent optimization on them. We chose *Adam* as the optimization algorithm. 

In [8]:
X = tf.constant(dtrain[['intercept', 'Var1', 'Var2', 'Var3']].values, name = 'X', dtype=tf.float32)
y = tf.constant(value = list(dtrain['resp']), dtype = tf.float32, name='y', shape=(dtrain.shape[0], 1))

w = tf.Variable(tf.zeros([4, 1]))

y_hat = tf.exp(tf.matmul(X, w))

loss_function = tf.reduce_mean(-y*tf.log(y_hat)+y_hat)

train_step = tf.train.AdamOptimizer(0.001).minimize(loss_function)
init = tf.global_variables_initializer()
session = tf.InteractiveSession()
session.run(init)

for i in range(10000):
    session.run(train_step)

/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


### The result 

In [9]:
w.eval()

array([[-2.0145135 ],
       [ 1.0417475 ],
       [-0.55138296],
       [ 0.34045687]], dtype=float32)

### Comparison with the statsmodels package 
To check the results, we repeat solving the same regression problem using the statsmodels library:

In [11]:
poisson_family = sm.families.Poisson()
poisson_model = sm.GLM(dtrain['resp'], dtrain[['intercept', 'Var1', 'Var2', 'Var3']], family=poisson_family)
poisson_results = poisson_model.fit()

In [12]:
poisson_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   resp   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9996
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -4487.3
Date:                Sat, 02 Jul 2022   Deviance:                       5862.7
Time:                        16:05:46   Pearson chi2:                 1.01e+04
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -2.0145      0.030    -67.132      0.000      -2.073      -1.956
Var1           1.0418      0.047     22.202      0.000       0.950       1.134
Var2          -0.5514      0.044    -12.652      0.000      -0.637      -0.466
Var3           0.3405      0.042      8.127      0.000       0.258       0.423
==============================================================================
"""